# Golden SAML AD FS Mail Access Hunt

#### Dataset
https://securitydatasets.com/notebooks/compound/GoldenSAMLADFSMailAccess.html
- WindowsEvents.Zip
- Microsoft365DefenderEvents.Zip
- AADAuditEvents.Zip
- OfficeActivityEvents.Zip

#### Simple Data Ingestion

All data is unzipped and loaded to two SQlite stores (SQL-supported lakehouse simulation) using [kestrel-tool::mkdb](https://github.com/opencybersecurityalliance/kestrel-lang/blob/develop/packages/kestrel_tool/src/kestrel_tool/mkdb.py)

#### Simulated On-premise Datastore `sqlite:///onpremise.db`

| Data File | Datastore Index | Kestrel Datasource | ........................................................................................................................................................................ |
| :------- | :------ | :------ | ------- |
| WindowsEvents.Zip | windows | GoldenSAML-WindowsEvents ||

#### Simulated Cloud Datastore `sqlite:///cloud.db`

| Data File | Datastore Index | Kestrel Datasource | .............................................................................................................................. |
| :------- | :------ | :------ | ------- |
| Microsoft365DefenderEvents.Zip | msdefender | GoldenSAML-Microsoft365DefenderEvents||
| AADAuditEvents.Zip | aad | GoldenSAML-AADAuditEvents ||
| OfficeActivityEvents.Zip | office | GoldenSAML-OfficeActivityEvents||

#### Analytics Ready to Use Besides Data Retrieval

- `rare-event-detection`: a Python analytics to find rare events from a pool of events. This is an example of hunt steps invoking Turing complete logic besides SQL to support computation like clustering and classification.

- `ask-AI`: a simulated LLM cyber agent to answer questions regarding specified fields in a Kestrel variable. The answer is enriched to the Kestrel variable as a new attribute (unmapped OCSF field) `unmapped.gen_ai`. A real OpenAI-API-based Kestrel analytics example can be found at [Kestrel analytics repo](https://github.com/opencybersecurityalliance/kestrel-analytics/tree/release/analytics/openai-suspicious-processes).

## Steps in This Huntbook

1. Start from Windows events to locate suspicious events (already written in this huntbook)
2. Think about potential campaigns, e.g., *GoldenSAML*, from the most suspicious Window event
3. Read the [GoldenSAML blog](https://www.cyberark.com/resources/threat-research-blog/golden-saml-newly-discovered-attack-technique-forges-authentication-to-cloud-apps) to understand the generic attack flow
4. Develop the threat hypothesis around the [SimuLand GoldenSAML attack simulation](https://simulandlabs.com/labs/GoldenSAML/README.html)
5. Move across multiple data sources to verify different phases of the attack from multiple angles (already written in this huntbook)
6. **Execute this huntbook and report your findings**
7. Further drill down
    - Add new cells in this huntbook to explore other paths of the hunt
    - Explore other aspects of the attack with any *goldensaml-quiz* huntbook
    - Check the *goldensaml-explain-kestrel* huntbook to get a basic idea of the Kestrel abstraction

In [1]:
# start from zero: detect rare events from a single day's Windows logs

all_events = GET event FROM sqlalchemy://GoldenSAML-WindowsEvents
             WHERE device.hostname LIKE '%'
             START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z

In [2]:
DISP all_events ATTR time, type_uid, type_name, device.hostname SORT BY time ASC

time,type_uid,type_name,device.hostname
2021-08-02T13:05:32.77Z,100114,Pipe connected,adfs01.simulandlabs.com
2021-08-02T13:05:33.483Z,33205,33205,adfs01.simulandlabs.com
2021-08-02T13:06:38.337Z,300201,An account was successfully logged on,dc01.simulandlabs.com
2021-08-02T13:06:38.33Z,300201,An account was successfully logged on,dc01.simulandlabs.com
2021-08-02T13:06:39.587Z,300201,An account was successfully logged on,dc01.simulandlabs.com
2021-08-02T13:06:40.033Z,300201,An account was successfully logged on,dc01.simulandlabs.com
2021-08-02T13:08:45.653Z,300201,An account was successfully logged on,dc01.simulandlabs.com
2021-08-02T13:08:45.68Z,300201,An account was successfully logged on,dc01.simulandlabs.com
2021-08-02T13:09:19.323Z,400101,The Windows Filtering Platform has permitted a connection,adfs01.simulandlabs.com
2021-08-02T13:09:20.04Z,412,412,adfs01.simulandlabs.com


In [3]:
rare_events = all_events

# this example analytics (Python function) will filter only rare events by counting
APPLY python://rare-event-detection ON rare_events

# more details can be provided to the analytics as parameters
# APPLY python://rare-event-detection ON rare_events WITH field='type_uid', threshold=5

DISP rare_events ATTR time, type_uid, type_name, device.hostname SORT BY time ASC

time,type_uid,type_name,device.hostname
2021-08-02T13:05:32.77Z,100114,Pipe connected,adfs01.simulandlabs.com
2021-08-02T13:05:33.483Z,33205,33205,adfs01.simulandlabs.com
2021-08-02T13:09:19.323Z,400101,The Windows Filtering Platform has permitted a connection,adfs01.simulandlabs.com
2021-08-02T13:09:20.04Z,412,412,adfs01.simulandlabs.com
2021-08-02T13:09:20.04Z,100799,Desktop Window Manager is experiencing heavy resource contention,adfs01.simulandlabs.com
2021-08-02T13:11:39.77Z,300400,An operation was performed on an object,dc01.simulandlabs.com
2021-08-02T13:11:53.63Z,300400,An operation was performed on an object,dc01.simulandlabs.com
2021-08-02T13:13:11.003Z,300400,An operation was performed on an object,dc01.simulandlabs.com
2021-08-02T13:13:11Z,300400,An operation was performed on an object,dc01.simulandlabs.com


In [4]:
# let's first explore the (the single) network connection event

conn_event = rare_events WHERE type_uid = 400101

src = FIND network_endpoint ORIGINATED conn_event
DISP src

dst = FIND network_endpoint RESPONDED conn_event
DISP dst

ip,port
fe80::a405:857d:204:efa5,53121.0


ip,port
fe80::a405:857d:204:efa5,80.0


In [5]:
# just a connection to localhost:80, nothing suspicious at the current moment

In [6]:
# next let's check the pipe event

pipe_event = rare_events WHERE type_uid = 100114

pipe = FIND file RESPONDED pipe_event
DISP pipe

endpoint.hostname,endpoint.uid,name,type_id
adfs01.simulandlabs.com,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,\MICROSOFT##WID\tsql\query,6


In [7]:
# leverage LLM's knowledge to make sense of the named pipe

APPLY python://ask-AI ON pipe WITH prompt='What is the following pipe in Windows?', field='name'

DISP pipe ATTR unmapped.gen_ai

unmapped.gen_ai
'\MICROSOFT##WID\tsql\query' is a path to connect to Windows Internal Database (WID). It can be used to export/exfiltrate AD FS configuration in a cyber attack


In [8]:
# which process created the pipe

pipe_proc = FIND process CREATED pipe

DISP pipe_proc

endpoint.hostname,endpoint.uid,name,file.name,file.path,file.parent_folder,pid,uid
adfs01.simulandlabs.com,/subscriptions/00000000-0000-0000-0000-000000000000/resourcegroups/azhybrid/providers/microsoft.compute/virtualmachines/adfs01,powershell.exe,powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0,5692,9acedb82-ccd4-6107-22de-030000000600


In [9]:
# this could be a GoldenSAML attack

# now it's time to read
# - https://www.cyberark.com/resources/threat-research-blog/golden-saml-newly-discovered-attack-technique-forges-authentication-to-cloud-apps
# - https://simulandlabs.com/labs/GoldenSAML/README.html

# questions:
# 1. do you find any relation between the pipe event and the connection event?
# 2. do we have more logs of the powershell from other angles/sources?

In [10]:
# let's check the suspicious process across another datasource: Microsoft365Defender

mde_events = GET event FROM sqlalchemy://GoldenSAML-Microsoft365DefenderEvents
             WHERE actor.process.pid = pipe_proc.pid
               AND actor.process.endpoint.hostname = pipe_proc.endpoint.hostname

DISP mde_events ATTR time, type_uid, type_name, device.hostname SORT BY time ASC

time,type_uid,type_name,device.hostname
2021-08-02T13:11:39.7724491Z,600504,LdapSearch,adfs01.simulandlabs.com
2021-08-02T13:11:39.7747609Z,600504,LdapSearch,adfs01.simulandlabs.com
2021-08-02T13:11:53.5836949Z,600504,LdapSearch,adfs01.simulandlabs.com
2021-08-02T13:11:53.6344905Z,600504,LdapSearch,adfs01.simulandlabs.com
2021-08-02T13:15:30.513652Z,600504,LdapSearch,adfs01.simulandlabs.com


In [11]:
# so what's searched in the LDAP searches

queries = FIND query_info RESPONDED mde_events

DISP queries

uid,attr_list,search_filter
113597,"[""objectClass""]",(objectClass=*)
113598,"[""thumbnailphoto""]",(&(objectclass=contact)(!name=CryptoPolicy)(ThumbnailPhoto=*))
113608,"[""""]",(name=CryptoPolicy)
113616,"[""thumbnailphoto""]",(l=9736f74f-fd37-4b02-80e8-8120a72ad6c2)
113771,"[""""]","(&(objectCategory=user)(memberOf=CN=Domain Admins,CN=Users,DC=simulandlabs,DC=com))"


In [12]:
# we found ________!!!

In [13]:
# looks like defender logs has lots of useful information, let's check other activities logged here

mde_all_events = GET event FROM sqlalchemy://GoldenSAML-Microsoft365DefenderEvents
                 WHERE type_name LIKE '%'
                 START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z

DISP mde_all_events ATTR time, type_uid, type_name SORT BY time ASC

time,type_uid,type_name
2021-08-02T13:08:18.374Z,600599,Directory Services replication
2021-08-02T13:11:39.7724491Z,600504,LdapSearch
2021-08-02T13:11:39.7747609Z,600504,LdapSearch
2021-08-02T13:11:53.5836949Z,600504,LdapSearch
2021-08-02T13:11:53.6344905Z,600504,LdapSearch
2021-08-02T13:13:02.057Z,600599,Directory Services replication
2021-08-02T13:15:30.513652Z,600504,LdapSearch
2021-08-02T13:27:20Z,300501,Add delegated permission grant.
2021-08-02T13:32:07Z,300403,MailItemsAccessed


In [14]:
# attack impact analysis

delegation_event = mde_all_events WHERE type_uid = 300501

DISP delegation_event

time,query_info.uid,entity.uid,entity.data,privileges,type_uid,type_name
2021-08-02T13:27:20Z,106830890_11161_e417c1fd-ae0b-4efa-891b-bb6b0ea7d35e,106830890_11161_e417c1fd-ae0b-4efa-891b-bb6b0ea7d35e,"[{""ApplicationInstance"": 0, ""ApplicationId"": 11161, ""Type"": ""User"", ""Role"": ""Actor"", ""Name"": ""Gustavo Pedro"", ""Id"": ""aead923d-498b-4f64-a66c-2af91447a8b6""}]",User.Read Mail.ReadWrite,300501,Add delegated permission grant.


In [15]:
mailaccess_event = mde_all_events WHERE type_uid = 300403

accessed_mail = FIND managed_entity RESPONDED mailaccess_event

DISP accessed_mail

uid,data
106830890_20893_699e0b10-1c53-403e-976f-ce0847a92b44,"[{""ServiceObjectType"": ""Session ID"", ""Type"": ""Structured object"", ""Role"": ""Parameter""}, {""Type"": ""Task"", ""Role"": ""Target object"", ""Name"": ""MailItemsAccessed""}, {""Type"": ""Property"", ""Role"": ""Parameter"", ""Name"": ""MailAccessType"", ""Value"": ""Bind""}, {""Type"": ""Property"", ""Role"": ""Parameter"", ""Name"": ""IsThrottled"", ""Value"": ""False""}, {""ApplicationInstance"": 0, ""ApplicationId"": 11161, ""Type"": ""User"", ""Role"": ""Parameter"", ""Name"": ""Gustavo Pedro"", ""Id"": ""aead923d-498b-4f64-a66c-2af91447a8b6""}, {""ApplicationInstance"": 0, ""ApplicationId"": 11161, ""Type"": ""Account"", ""Role"": ""Actor"", ""Name"": ""SimuLandApp"", ""Id"": ""5a95e683-08ad-424e-a441-1d1aec52c02c""}]"


In [16]:
# attack impact analysis and confirmation with data from other datasources

aad_events = GET event FROM sqlalchemy://GoldenSAML-AADAuditEvents
             WHERE type_name LIKE '%'
             START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z

DISP aad_events ATTR time, type_uid, type_name SORT BY time ASC

time,type_uid,type_name
2021-08-02T13:25:12.246Z,300403,Update application
2021-08-02T13:27:20.017Z,300501,Add delegated permission grant
2021-08-02T13:29:25.983Z,300403,Update application – Certificates and secrets management


In [17]:
DISP aad_events SORT BY time ASC

time,device.uid,actor.user.endpoint.uid,http_request.user_agent,entity.uid,entity.type,entity.name,entity.data,actor.user.uid,actor.user.name,privileges,type_uid,type_name,status_id
2021-08-02T13:25:12.246Z,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000,Mozilla/5.0 (Windows NT; Windows NT 10.0; en-US) WindowsPowerShell/5.1.17763.1971,11b49e19-2326-4be6-93cb-7f37439bbd81,Application,SimuLandApp,"[{""id"": ""11b49e19-2326-4be6-93cb-7f37439bbd81"", ""displayName"": ""SimuLandApp"", ""type"": ""Application"", ""modifiedProperties"": [{""displayName"": ""RequiredResourceAccess"", ""oldValue"": ""[{\""ResourceAppId\"":\""00000003-0000-0000-c000-000000000000\"",\""RequiredAppPermissions\"":[{\""EntitlementId\"":\""e1fe6dd8-ba31-4d61-89e7-88639da4683d\"",\""DirectAccessGrant\"":false,\""ImpersonationAccessGrants\"":[20]}],\""EncodingVersion\"":1}]"", ""newValue"": ""[{\""ResourceAppId\"":\""00000003-0000-0000-c000-000000000000\"",\""RequiredAppPermissions\"":[{\""EntitlementId\"":\""e1fe6dd8-ba31-4d61-89e7-88639da4683d\"",\""DirectAccessGrant\"":false,\""ImpersonationAccessGrants\"":[20]},{\""EntitlementId\"":\""024d486e-b451-40bb-833d-3e66d98c5c73\"",\""DirectAccessGrant\"":false,\""ImpersonationAccessGrants\"":[20]}],\""EncodingVersion\"":1}]""}, {""displayName"": ""Included Updated Properties"", ""oldValue"": null, ""newValue"": ""\""RequiredResourceAccess\""""}], ""administrativeUnits"": []}]",aead923d-498b-4f64-a66c-2af91447a8b6,pgustavo@simulandlabs.com,None,300403,Update application,1
2021-08-02T13:27:20.017Z,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000,Mozilla/5.0 (Windows NT; Windows NT 10.0; en-US) WindowsPowerShell/5.1.17763.1971,None,ServicePrincipal,None,"[{""id"": ""401dd906-ea4f-4d41-b762-7e936d222368"", ""displayName"": ""Microsoft Graph"", ""type"": ""ServicePrincipal"", ""modifiedProperties"": [{""displayName"": ""DelegatedPermissionGrant.Scope"", ""oldValue"": ""\""User.Read\"""", ""newValue"": ""\""User.Read Mail.ReadWrite\""""}, {""displayName"": ""DelegatedPermissionGrant.ConsentType"", ""oldValue"": ""\""AllPrincipals\"""", ""newValue"": ""\""AllPrincipals\""""}, {""displayName"": ""ServicePrincipal.ObjectID"", ""oldValue"": null, ""newValue"": ""\""0d2f5969-011b-460d-ac74-3291d227d49f\""""}, {""displayName"": ""ServicePrincipal.DisplayName"", ""oldValue"": null, ""newValue"": null}, {""displayName"": ""ServicePrincipal.AppId"", ""oldValue"": null, ""newValue"": null}, {""displayName"": ""ServicePrincipal.Name"", ""oldValue"": null, ""newValue"": null}, {""displayName"": ""TargetId.ServicePrincipalNames"", ""oldValue"": null, ""newValue"": ""\""00000003-0000-0000-c000-000000000000/ags.windows.net;00000003-0000-0000-c000-000000000000;https://canary.graph.microsoft.com;https://graph.microsoft.com;https://ags.windows.net;https://graph.microsoft.us;https://graph.microsoft.com/;https://dod-graph.microsoft.us;https://canary.graph.microsoft.com/;https://graph.microsoft.us/;https://dod-graph.microsoft.us/\""""}], ""administrativeUnits"": []}, {""id"": ""0d2f5969-011b-460d-ac74-3291d227d49f"", ""displayName"": null, ""type"": ""ServicePrincipal"", ""modifiedProperties"": [], ""administrativeUnits"": []}]",aead923d-498b-4f64-a66c-2af91447a8b6,pgustavo@simulandlabs.com,User.Read Mail.ReadWrite,300501,Add delegated permission grant,1
2021-08-02T13:29:25.983Z,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000,Mozilla/5.0 (Windows NT; Windows NT 10.0; en-US) WindowsPowerShell/5.1.17763.1971,11b49e19-2326-4be6-93cb-7f37439bbd81,Application,SimuLandApp,"[{""id"": ""11b49e19-2326-4be6-93cb-7f37439bbd81"", ""displayName"": ""SimuLandApp"", ""type"": ""Application"", ""modifiedProperties"": [{""displayName"": ""KeyDescription"", ""oldValue"": ""[]"", ""newValue"": ""[\""[KeyIdentifier=59eaeebc-8a6b-44a6-9f24-7d55e64420c9,KeyType=Password,KeyUsage=Verify,DisplayName=SimuLandCreds]\""]""}, {""displayName"": ""Included Updated Properties"", ""oldValue"": n

In [18]:
# attack impact analysis and confirmation with data from other datasources

office_events = GET event FROM sqlalchemy://GoldenSAML-OfficeActivityEvents
                WHERE type_name = mailaccess_event.type_name
                START 2021-08-02T00:00:00.000Z STOP 2021-08-03T00:00:00.000Z

DISP office_events

time,device.uid,actor.user.endpoint.uid,src_endpoint.ip,entity.uid,entity.type,entity.data,actor.user.uid,actor.user.name,type_uid,type_name,status_id
2021-08-02T13:32:07Z,00000000-0000-0000-0000-000000000000,00000000-0000-0000-0000-000000000000,1.2.3.4,d0c5f8ae-9ed7-4e46-bfdf-ea1460f5a31b,OfficeActivity,"[{""FolderItems"": [{""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.centralus.production.microsoft.com>""}, {""InternetMessageId"": ""<AB1CD23EF4567DAC24353B8AD047DDEB0B1EB9@AB1CD23EF4567.abcdef12.prod.outlook.com>""}, {""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.southcentralus.production.microsoft.com>""}, {""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.eastus.production.microsoft.com>""}, {""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.northcentralus.production.microsoft.com>""}, {""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.northcentralus.production.microsoft.com>""}, {""InternetMessageId"": ""<00000000-0000-0000-0000-000000000000@az.westus2.production.microsoft.com>""}], ""Id"": ""LgAAAAAM7KyTTmWeRac2KXBEz/7aAQARGHK+grzLTpRJraC1QR6kAAAAAAEMAAAB"", ""Path"": ""\\Inbox""}]",S-1-5-21-1825954961-3338807533-2873504967-26087451,pgustavo@simulandlabs.com,300403,MailItemsAccessed,1
